# Generating the supplemental data file
Concatenate cross-validation and ensemble prediction results into a single CSV

In [1]:
from ysi_utils.data import low, addl_mols
from ysi_utils.descriptors import dragon
from ysi_utils.validation import y_train as train
from ysi_utils.validation import y_test

In [2]:
train['Group'] = 'train'
y_test['Group'] = 'test'
addl_mols['Group'] = 'experiment'

In [3]:
# Add the new molecules to the test set
raddl_mols = addl_mols.rename(columns={'Name': 'Species'})
raddl_mols['Type'] = 'New Compounds'
raddl_mols['Ref'] = 0
test = y_test.append(raddl_mols.set_index('SMILES'))

X = dragon.loc[train.index]
X_test = dragon.loc[test.index]

In [4]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    from ysi_utils.models import outlier_model, bagging_model, ensemble_predict
    ensemble_predict.bagging_model = bagging_model()

Using Theano backend.


In [5]:
y_test_predictions = ensemble_predict(X_test)
y_train_predictions = ensemble_predict(X.values)

In [6]:
import pandas as pd

y_cv_df = pd.read_pickle('y_cv_random_results.p')

In [7]:
train['YSI_pred_mean'] = y_train_predictions.mean(1)
train['YSI_pred_std'] = y_train_predictions.std(1)

train['YSI_cv_mean'] = y_cv_df.mean(1)
train['YSI_cv_std'] = y_cv_df.std(1)

test['YSI_pred_mean'] = y_test_predictions.mean(1)
test['YSI_pred_std'] = y_test_predictions.std(1)

In [8]:
train.append(test).reset_index().to_csv('prediction_summary.csv', float_format='%.2f')